In [54]:
 #@title Temp and ignore
#creates train_dataset
import pandas as pd
train = []
data=[]
tags=[]
text = True
fr = open("labelled_data.txt")
for i in fr.readlines():
  if len(i)==0:
    continue
  else:
    #print(i)
    if text:
      data = i.split()
      text = not text
    else:
      tags = i.split()
      #print(data,tags)
      tmp = [data,tags]
      train.append(tuple(tmp))
      text = not text
test = train
tags_vals = []
for tag in train:
  tags_vals.extend(tag[1])
tags_vals = list(set(tags_vals))
tag2id = {t: i for i, t in enumerate(tags_vals)}
print(tag2id)

{'T007': 0, 'T047': 1, '0': 2}


In [93]:
!pip install pytorch_transformers flair seqeval

In [0]:
#!mkdir -p conll2003
#!wget https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.testa -O ./conll2003/eng.testa
#!wget https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.testb -O ./conll2003/eng.testb
#!wget https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.train -O ./conll2003/eng.train

In [0]:
import os
import logging
import sys
import torch

In [0]:
# Choosing cuda devices if there are multiple
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [0]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger('sequence_tagger_bert')

In [98]:
device = torch.device('cuda')
n_gpu = torch.cuda.device_count()

for i in range(n_gpu):
    print(torch.cuda.get_device_name(i))

Tesla P4


In [0]:
CACHE_DIR = 'cache'
BATCH_SIZE = 16
PRED_BATCH_SIZE = 100
MAX_LEN = 128
MAX_N_EPOCHS = 4
WEIGHT_DECAY = 0.01
LEARNING_RATE = 5e-5

In [100]:
torch.manual_seed(117)

In [0]:
#from flair.datasets import ColumnCorpus
#data_folder = 'conll2003'
#corpus = ColumnCorpus(data_folder, 
#                      {0 : 'text', 3 : 'ner'},
#                      train_file='eng.train',
#                      test_file='eng.testb',
#                      dev_file='eng.testa')
#
#print(corpus.obtain_statistics())
#print(corpus.dev[1].to_tagged_string('ner'))

In [102]:
from flair.datasets import ColumnCorpus
data_folder = 'conll2003'
corpus = ColumnCorpus(data_folder, 
                      {0 : 'idx', 2 : 'text', 3 : 'ner' },
                      train_file='dataset.txt',
                      test_file='dataset.txt',
                      dev_file='dataset.txt')
print(corpus.train[1].to_tagged_string('ner'))

2020-03-21 14:31:48,838 Reading data from conll2003
2020-03-21 14:31:48,840 Train: conll2003/dataset.txt
2020-03-21 14:31:48,842 Dev: conll2003/dataset.txt
2020-03-21 14:31:48,843 Test: conll2003/dataset.txt
Pseudomonas <T047> aeruginosa <T047> (Pa) <T047> infection <T047> in cystic <T047> fibrosis <T047> (CF) patients is associated with worse long-term pulmonary <T047> disease <T047> and shorter "survival," and chronic <T047> Pa <T047> infection <T047> (CPA) is associated with reduced lung "function," faster rate of lung "decline," increased rates of exacerbations and shorter survival


In [103]:
from bert_sequence_tagger import SequenceTaggerBert, BertForTokenClassificationCustom
from bert_sequence_tagger.bert_utils import make_bert_tag_dict_from_flair_corpus

from pytorch_transformers import BertTokenizer, BertForTokenClassification


bpe_tokenizer = BertTokenizer.from_pretrained('bert-base-cased', 
                                              cache_dir=CACHE_DIR, 
                                              do_lower_case=False)

idx2tag, tag2idx = make_bert_tag_dict_from_flair_corpus(corpus)

model = BertForTokenClassificationCustom.from_pretrained('bert-base-cased', 
                                                         cache_dir=CACHE_DIR, 
                                                         num_labels=len(tag2idx)).cuda()

seq_tagger = SequenceTaggerBert(bert_model=model, bpe_tokenizer=bpe_tokenizer, 
                                idx2tag=idx2tag, tag2idx=tag2idx, max_len=MAX_LEN,
                                pred_batch_size=PRED_BATCH_SIZE)

INFO:pytorch_transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at cache/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
INFO:pytorch_transformers.modeling_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json from cache at cache/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.3d5adf10d3445c36ce131f4c6416aa62e9b58e1af56b97664773f4858a46286e
INFO:pytorch_transformers.modeling_utils:Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers":

In [104]:
print(idx2tag,tag2idx)

['[PAD]', 'O', 'T047', '"T019,T047"', 'T007', 'T005', 'T016', '"T007,T204"', '"T047,T190"'] {'[PAD]': 0, 'O': 1, 'T047': 2, '"T019,T047"': 3, 'T007': 4, 'T005': 5, 'T016': 6, '"T007,T204"': 7, '"T047,T190"': 8}


In [105]:
from bert_sequence_tagger.bert_utils import get_model_parameters, prepare_flair_corpus
from bert_sequence_tagger.model_trainer_bert import ModelTrainerBert
from bert_sequence_tagger.metrics import f1_entity_level, f1_token_level

from pytorch_transformers import AdamW, WarmupLinearSchedule


train_dataset = prepare_flair_corpus(corpus.train)
val_dataset = prepare_flair_corpus(corpus.dev)
print(train_dataset)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
optimizer = AdamW(get_model_parameters(model), 
                  lr=LEARNING_RATE, betas=(0.9, 0.999), 
                  eps =1e-6, weight_decay=0.01, correct_bias=True)
lr_scheduler = WarmupLinearSchedule(optimizer, warmup_steps=0.1, 
                                    t_total=(len(corpus.train) / BATCH_SIZE)*MAX_N_EPOCHS)

In [107]:
trainer = ModelTrainerBert(model=seq_tagger, 
                           optimizer=optimizer, 
                           lr_scheduler=lr_scheduler,
                           train_dataset=train_dataset, 
                           val_dataset=val_dataset,
                           update_scheduler='es',
                           validation_metrics=[f1_entity_level],
                           batch_size=BATCH_SIZE)

trainer.train(epochs=MAX_N_EPOCHS)

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

INFO:sequence_tagger_bert:Current learning rate: 3.74959022818945e-05
INFO:sequence_tagger_bert:Train loss: 0.04333878487130632
INFO:sequence_tagger_bert:Validation loss: 0.018547681014544626
INFO:sequence_tagger_bert:Validation metrics: (0.747334371630526,)


Epoch:  25%|██▌       | 1/4 [19:14<57:43, 1154.44s/it]

INFO:sequence_tagger_bert:Current learning rate: 2.4991329466037638e-05
INFO:sequence_tagger_bert:Train loss: 0.017644170739244545
INFO:sequence_tagger_bert:Validation loss: 0.008327192969572112
INFO:sequence_tagger_bert:Validation metrics: (0.8886273190964632,)


Epoch:  50%|█████     | 2/4 [38:32<38:30, 1155.43s/it]

INFO:sequence_tagger_bert:Current learning rate: 1.2486756650180776e-05
INFO:sequence_tagger_bert:Train loss: 0.008455805816446569
INFO:sequence_tagger_bert:Validation loss: 0.0038848596508130806
INFO:sequence_tagger_bert:Validation metrics: (0.9411634716314016,)


Epoch:  75%|███████▌  | 3/4 [57:52<19:16, 1156.81s/it]

INFO:sequence_tagger_bert:Current learning rate: 0.0
INFO:sequence_tagger_bert:Train loss: 0.0035580829734531065
INFO:sequence_tagger_bert:Validation loss: 0.0013199315755666936
INFO:sequence_tagger_bert:Validation metrics: (0.9794413407821227,)


Epoch: 100%|██████████| 4/4 [1:17:12<00:00, 1158.21s/it]


In [108]:
test_dataset = prepare_flair_corpus(corpus.test)

_, __, test_metrics = seq_tagger.predict(test_dataset, evaluate=True, 
                                         metrics=[f1_entity_level, f1_token_level])
logger.info(f'Entity-level f1: {test_metrics[1]}')
logger.info(f'Token-level f1: {test_metrics[2]}')

INFO:sequence_tagger_bert:Entity-level f1: 0.9794413407821227
INFO:sequence_tagger_bert:Token-level f1: 0.9891479531535404


In [109]:
seq_tagger.predict([['We', 'are', 'living', 'in', 'New', 'York', 'city', '.'],
                    ['Satya', 'Narayana', 'Nadella', 'is', 'an', 'engineer', 'and', 'business', 'executive', '.']])

([['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']],
 [10.016705, 8.675538])

In [0]:
seq_tagger.save_serialize('./bert_model')

In [119]:
check1 = ("Chikungunya	virus	(	CHIKV	)	is	a	mosquito").split()
check = ("DCTN4	as	a	modifier	of	chronic	Pseudomonas	aeruginosa	infection	in	cystic	fibrosis.").split()
print(check)
seq_tagger.predict([check])

['DCTN4', 'as', 'a', 'modifier', 'of', 'chronic', 'Pseudomonas', 'aeruginosa', 'infection', 'in', 'cystic', 'fibrosis.']


([['O',
   'O',
   'O',
   'O',
   'O',
   'T047',
   'T047',
   'T047',
   'T047',
   'O',
   'T047',
   'T047']],
 [8.894433])